<a href="https://colab.research.google.com/github/cdiegor/MineracaoDeDados/blob/main/Pr%C3%A1tica_1_Introdu%C3%A7%C3%A3o_%C3%A0_Minera%C3%A7%C3%A3o_de_Dados_Fluxo_e_Valida%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Prática 1 - Fluxo de trabalho e validação de modelos

**Disciplina:** Introdução à Mineração de Dados  
**Foco da prática:** Entradas/saídas, ciclo de dados, particionamento (treino/validação/teste), métricas de avaliação, validação cruzada, curvas de aprendizado.



In [ ]:

# --- Preparação ------------------------------------------------------------------
# Se necessário, instale scikit-learn (em muitos ambientes educacionais já está instalado).
# !pip install scikit-learn pandas numpy matplotlib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import load_breast_cancer, fetch_california_housing
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, KFold, learning_curve
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, ConfusionMatrixDisplay,
    RocCurveDisplay, PrecisionRecallDisplay,
    mean_squared_error, r2_score, brier_score_loss
)
from sklearn.calibration import calibration_curve
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.utils import shuffle

np.random.seed(7)



## 1. Fluxo de ML & divisões de dados

**Etapas típicas**: formulação do problema → coleta/limpeza/EDA → engenharia de atributos → modelagem → avaliação & validação → seleção → implantação/monitoramento.

**Divisões**:
- **Treino**: ajustar modelos e fazer tuning de hiperparâmetros (com CV no conjunto de treino).
- **Validação**: (opcional quando não se usa CV) seleção preliminar de modelos; em CV aninhada, o laço interno decide.
- **Teste**: uso **único** ao final, para estimativa imparcial.

> **Regra de ouro**: Nunca usar o conjunto de teste para tomar decisões de modelo/atributos.



## 2. Demonstração de classificação — Baseline, métricas e validação cruzada

Usaremos o conjunto **Breast Cancer Wisconsin** (classificação binária).


In [ ]:

# Carregar dados
data = load_breast_cancer(as_frame=True)
X = data.data
y = pd.Series(data.target, name="alvo")
X.head()


In [ ]:

# Split treino/teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

print(X_train.shape, X_test.shape)
y_train.value_counts(normalize=True).round(3)


In [ ]:

# Modelo baseline: Regressão Logística com padronização
pipe_lr = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(max_iter=200))
])

# Validação cruzada no conjunto de treino
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(pipe_lr, X_train, y_train, cv=cv, scoring="roc_auc")
print("CV ROC-AUC:", np.round(cv_scores, 4), "Média:", cv_scores.mean().round(4))


In [ ]:

# Ajustar e avaliar no conjunto de teste
pipe_lr.fit(X_train, y_train)
proba = pipe_lr.predict_proba(X_test)[:, 1]
pred = (proba >= 0.5).astype(int)

acc = accuracy_score(y_test, pred)
prec = precision_score(y_test, pred)
rec = recall_score(y_test, pred)
f1 = f1_score(y_test, pred)
auc = roc_auc_score(y_test, proba)

print(f"Teste -> Acurácia={acc:.3f}  Precisão={prec:.3f}  Revocação={rec:.3f}  F1={f1:.3f}  ROC-AUC={auc:.3f}")


In [ ]:

# Matriz de confusão e curvas ROC/PR
cm = confusion_matrix(y_test, pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.title("Matriz de Confusão (limiar=0.5)")
plt.show()

RocCurveDisplay.from_predictions(y_test, proba)
plt.title("Curva ROC")
plt.show()

PrecisionRecallDisplay.from_predictions(y_test, proba)
plt.title("Curva Precisão-Revocação")
plt.show()



## 3. Curvas de aprendizado & intuição viés–variância

Curvas de aprendizado mostram o desempenho de treino e de validação em função do tamanho do conjunto de treino.


In [ ]:

train_sizes, train_scores, valid_scores = learning_curve(
    pipe_lr, X_train, y_train, cv=cv, scoring="roc_auc", train_sizes=np.linspace(0.1, 1.0, 6), n_jobs=None
)

train_mean = train_scores.mean(axis=1)
valid_mean = valid_scores.mean(axis=1)

plt.figure()
plt.plot(train_sizes, train_mean, marker='o', label="ROC-AUC Treino")
plt.plot(train_sizes, valid_mean, marker='s', label="ROC-AUC Validação (CV)")
plt.xlabel("Exemplos de treino")
plt.ylabel("Pontuação")
plt.title("Curva de Aprendizado (Reg. Logística)")
plt.legend()
plt.grid(True)
plt.show()



## 4. Calibração de probabilidades

Boas probabilidades são úteis (ajuste de limiares, decisões custo-sensíveis). Vamos inspecionar o **gráfico de calibração** e o **Brier score**.


In [ ]:

prob_true, prob_pred = calibration_curve(y_test, proba, n_bins=10, strategy="quantile")
brier = brier_score_loss(y_test, proba)
print(f"Brier score = {brier:.4f} (menor é melhor)")

plt.figure()
plt.plot(prob_pred, prob_true, marker='o', label="Modelo")
plt.plot([0,1], [0,1], linestyle='--', label="Perfeitamente calibrado")
plt.xlabel("Probabilidade prevista")
plt.ylabel("Frequência observada")
plt.title("Curva de Calibração")
plt.legend()
plt.grid(True)
plt.show()



## 5. Desbalanceamento de classes & limiares

Desbalanceamento afeta as métricas. Vamos simular um desbalanceamento mais forte reduzindo a classe positiva no treino e comparar.


In [ ]:

# Treino desbalanceado (apenas para demonstração)
X_tr_imb, y_tr_imb = X_train.copy(), y_train.copy()
pos_idx = y_tr_imb[y_tr_imb == 1].index
keep_pos = np.random.choice(pos_idx, size=max(10, len(pos_idx)//5), replace=False)
mask = y_tr_imb.index.isin(keep_pos) | (y_tr_imb == 0)
X_tr_imb = X_tr_imb.loc[mask]
y_tr_imb = y_tr_imb.loc[mask]

pipe_lr_imb = Pipeline([("scaler", StandardScaler()), ("clf", LogisticRegression(max_iter=200, class_weight=None))])
pipe_lr_imb.fit(X_tr_imb, y_tr_imb)
proba_imb = pipe_lr_imb.predict_proba(X_test)[:, 1]

print("AUC do modelo padrão:", roc_auc_score(y_test, proba))
print("AUC do modelo treinado desbalanceado:", roc_auc_score(y_test, proba_imb))


In [ ]:

# Exemplo de ajuste de limiar no modelo padrão
thresholds = np.linspace(0.1, 0.9, 9)
records = []
for thr in thresholds:
    pred_thr = (proba >= thr).astype(int)
    records.append({
        "limiar": thr,
        "precisao": precision_score(y_test, pred_thr),
        "revocacao": recall_score(y_test, pred_thr),
        "f1": f1_score(y_test, pred_thr)
    })

thr_df = pd.DataFrame(records)
thr_df



## 6. Mini-demo de regressão — Métricas e validação

Usaremos **California Housing** para ilustrar métricas de regressão e validação.


In [ ]:

house = fetch_california_housing(as_frame=True)
Xr = house.data
yr = house.target

Xr_train, Xr_test, yr_train, yr_test = train_test_split(Xr, yr, test_size=0.2, random_state=42)

pipe_lin = Pipeline([("scaler", StandardScaler(with_mean=False)), ("reg", LinearRegression())])
# with_mean=False evita problemas caso matrizes esparsas sejam usadas (aqui é denso).

kfold = KFold(n_splits=5, shuffle=True, random_state=42)
cv_mse = -cross_val_score(pipe_lin, Xr_train, yr_train, cv=kfold, scoring="neg_mean_squared_error")
print("CV RMSE:", np.sqrt(cv_mse).round(3), "Média:", np.sqrt(cv_mse.mean()).round(3))

pipe_lin.fit(Xr_train, yr_train)
pred_r = pipe_lin.predict(Xr_test)
rmse = np.sqrt(mean_squared_error(yr_test, pred_r))
r2 = r2_score(yr_test, pred_r)
print(f"Teste -> RMSE={rmse:.3f}  R2={r2:.3f}")


In [ ]:

# Gráfico de resíduos
resid = yr_test - pred_r
plt.figure()
plt.scatter(pred_r, resid, alpha=0.6)
plt.axhline(0, linestyle='--')
plt.xlabel("Predito")
plt.ylabel("Resíduo (y - y_hat)")
plt.title("Gráfico de Resíduos")
plt.grid(True)
plt.show()



## 7. Um fluxo mínimo de análise de erros

1) Escolha uma métrica-alvo clara.  
2) Fatie o desempenho por coortes relevantes (ex.: tamanho, região, tempo).  
3) Inspecione exemplos de alto erro (FP/FN).  
4) Hipotetize atributos/transformações úteis; rode pequenas modificações.  
5) Documente o que funcionou e o que não funcionou (reprodutibilidade).
